In [16]:
!wget https://www.stats.govt.nz/assets/Uploads/Effects-of-COVID-19-on-trade/Download-data/Effects-of-COVID19-on-trade-1-February-03-June-2020-Provisional.csv

--2020-06-10 11:36:15--  https://www.stats.govt.nz/assets/Uploads/Effects-of-COVID-19-on-trade/Download-data/Effects-of-COVID19-on-trade-1-February-03-June-2020-Provisional.csv
Resolving www.stats.govt.nz (www.stats.govt.nz)... 45.60.11.104
Connecting to www.stats.govt.nz (www.stats.govt.nz)|45.60.11.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2376490 (2.3M) [text/csv]
Saving to: ‘Effects-of-COVID19-on-trade-1-February-03-June-2020-Provisional.csv’

Effects-of-COVID19- 100%[===================>]   2.27M  1.09MB/s    in 2.1s    

2020-06-10 11:36:19 (1.09 MB/s) - ‘Effects-of-COVID19-on-trade-1-February-03-June-2020-Provisional.csv’ saved [2376490/2376490]



In [17]:
#!pip install --quiet pyspark
from pyspark.sql import SparkSession, SQLContext, Row

spark = SparkSession.builder.appName("kdd").getOrCreate()
sc = spark.sparkContext
data_file = "/content/Effects-of-COVID19-on-trade-1-February-03-June-2020-Provisional.csv"
raw_rdd = sc.textFile(data_file).cache()
raw_rdd.take(5)

['Direction,Year,Date,Weekday,Current_Match,Country,Commodity,Transport_Mode,Measure,Value,Cumulative',
 'Exports,2015,31/01/2015,Saturday,01/02/2020,All,All,All,$,257000000,257000000',
 'Exports,2015,01/02/2015,Sunday,02/02/2020,All,All,All,$,123000000,380000000',
 'Exports,2015,02/02/2015,Monday,03/02/2020,All,All,All,$,176000000,556000000',
 'Exports,2015,03/02/2015,Tuesday,04/02/2020,All,All,All,$,115000000,671000000']

In [49]:
csv_rdd = raw_rdd.map(lambda row: row.split(","))
parsed_rdd = csv_rdd.map(lambda r: Row(
    Direction=r[0], 
    Year=r[1],
    Date=r[2],
    Weekday=r[3],
    Current_Match=r[4],
    Country=r[5],
    Commodity=r[6],
    Transport_Mode=r[7],
    Measure=r[8],
    Value=r[9],
    Cumulative=r[10]
    ))
parsed_rdd.take(2)

[Row(Commodity='Commodity', Country='Country', Cumulative='Cumulative', Current_Match='Current_Match', Date='Date', Direction='Direction', Measure='Measure', Transport_Mode='Transport_Mode', Value='Value', Weekday='Weekday', Year='Year'),
 Row(Commodity='All', Country='All', Cumulative='257000000', Current_Match='01/02/2020', Date='31/01/2015', Direction='Exports', Measure='$', Transport_Mode='All', Value='257000000', Weekday='Saturday', Year='2015')]

In [36]:
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(parsed_rdd)
connections_by_protocol = df.filter(df.Year!='Year').groupBy('Year').count().orderBy('count', ascending=False)
connections_by_protocol.show()

+----+-----+
|Year|count|
+----+-----+
|2015| 4708|
|2019| 4704|
|2018| 4702|
|2017| 4701|
|2016| 4699|
|2020| 4675|
+----+-----+



In [47]:
df.registerTempTable("connections")
attack_stats = sqlContext.sql("""
                           SELECT Country, year, count(year) as count
                           FROM connections
                           WHERE Country not in ("All", "Country")
                           GROUP BY Country, year
                           ORDER BY country, year
                           """)
attack_stats.show(truncate=False)


+---------------------------+----+-----+
|Country                    |year|count|
+---------------------------+----+-----+
|Australia                  |2015|248  |
|Australia                  |2016|248  |
|Australia                  |2017|248  |
|Australia                  |2018|248  |
|Australia                  |2019|248  |
|Australia                  |2020|248  |
|China                      |2015|1238 |
|China                      |2016|1237 |
|China                      |2017|1237 |
|China                      |2018|1236 |
|China                      |2019|1238 |
|China                      |2020|1215 |
|East Asia (excluding China)|2015|248  |
|East Asia (excluding China)|2016|248  |
|East Asia (excluding China)|2017|248  |
|East Asia (excluding China)|2018|248  |
|East Asia (excluding China)|2019|248  |
|East Asia (excluding China)|2020|248  |
|European Union (28)        |2015|124  |
|European Union (28)        |2016|124  |
+---------------------------+----+-----+
only showing top